In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(42)

In [ ]:
# ===============================
# STEP 1 — Parameters
# ===============================
weeks = 20
start_date = "2024-01-01"
freq = "W"  # weekly
regions = ["England", "Scotland", "Wales"]
category_name = "Chocolate Bar"

# 4 existing Dairy Milk variants (from Week 1)
# New product launches at Week 10 and has the SAME base price as "Dairy Milk Classic"
skus = {
    "Dairy Milk Classic 45g":      {"base_lambda": 220, "base_price": 1.09},
    "Dairy Milk Fruit & Nut 45g":  {"base_lambda": 160, "base_price": 1.39},
    "Dairy Milk Caramel 45g":      {"base_lambda": 140, "base_price": 1.29},
    "Dairy Milk Almond 45g":       {"base_lambda": 120, "base_price": 1.19},
}
new_sku_name = "New Dairy Milk Crunch 45g"
new_sku = {"base_lambda": 130, "launch_week": 10}  # launch in Week 10 (1-indexed)

# Pricing & promo behaviour
promo_probability = 0.27                # 27% of weeks on promo per SKU
promo_price_discount = 0.83             # 17% discount during promo (price *= 0.83)
promo_demand_lift = 1.39                # +29% demand when on promo (baseline effect)
price_elasticity = -1.3                 # elasticity: demand ~ (price/base_price)^elasticity

# Region multipliers (to create subtle geographic differences)
region_multipliers = {"England": 1.00, "Scotland": 0.83, "Wales": 0.67}

# Cannibalisation assumptions (applied AFTER the new SKU launches)
# For each unit of NEW sold, reduce Classic by 0.5 units and Caramel by 0.1 units
cannibalise_on = {
    "Dairy Milk Classic 45g": 0.38,
    "Dairy Milk Caramel 45g": 0.28,
    "Dairy Milk Fruit & Nut 45g": 0.13
    # "Dairy Milk Fruit & Nut 45g": 0.05,
    # "Dairy Milk Almond 45g": 0.05,
}


In [4]:
# ===============================
# STEP 2 — Date index
# ===============================
dates = pd.date_range(start=start_date, periods=weeks, freq=freq)

# A gentle seasonality curve for flavour (works fine even for 20 weeks)
# Peaks mid-period, dips at ends
theta = np.linspace(0, np.pi, weeks)
seasonality = 1.0 + 0.08 * np.sin(theta)  # ±8%

In [5]:
# ===============================
# STEP 3 — Helper functions
# ===============================
def sample_sales(expected_units: float) -> int:
    """Sample integer sales from a Poisson with the given mean, lower bounded at 0."""
    mean = max(0.0, expected_units)
    return int(np.random.poisson(lam=mean))

def compute_price_and_lift(base_price: float, on_promo: bool):
    """Return (price, demand_multiplier) given promo state."""
    if on_promo:
        price = round(base_price * promo_price_discount, 2)
        return price, promo_demand_lift
    return round(base_price, 2), 1.0

In [6]:
# ===============================
# STEP 4 — Generate rows
# ===============================
rows = []

# Pre-calculate base prices so the NEW SKU can share Classic's price
classic_price = skus["Dairy Milk Classic 45g"]["base_price"]
new_sku_base_price = classic_price  # same price as Dairy Milk Classic

for w_idx, date in enumerate(dates, start=1):
    # Identify if NEW is launched this week or later
    new_is_live = (w_idx >= new_sku["launch_week"])
    
    for region in regions:
        r_mult = region_multipliers[region]
        
        # --- Step 4a: First compute raw (pre-cannibalisation) sales for existing SKUs ---
        raw_sales = {}
        sku_prices = {}
        sku_promo_flags = {}
        
        for sku_name, cfg in skus.items():
            base_lambda = cfg["base_lambda"]
            base_price = cfg["base_price"]
            
            # Promo flag for this sku-week-region
            on_promo = (np.random.rand() < promo_probability)
            price, promo_lift = compute_price_and_lift(base_price, on_promo)
            
            # Expected demand (pre-cannibalisation)
            expected = base_lambda * seasonality[w_idx - 1] * r_mult
            # Apply price elasticity vs base price
            expected *= (price / base_price) ** price_elasticity
            # Promo lift
            expected *= promo_lift
            # Add a bit of random variance (10% lognormal noise)
            expected *= np.random.lognormal(mean=0, sigma=0.10)
            
            units = sample_sales(expected)
            
            raw_sales[sku_name] = units
            sku_prices[sku_name] = price
            sku_promo_flags[sku_name] = int(on_promo)
        
        # --- Step 4b: Compute NEW SKU (if live) ---
        new_units = 0
        new_price = None
        new_promo = 0
        
        if new_is_live:
            # Promo flag for new sku
            new_promo = int(np.random.rand() < promo_probability)
            new_price, new_lift = compute_price_and_lift(new_sku_base_price, bool(new_promo))
            
            expected_new = new_sku["base_lambda"] * seasonality[w_idx - 1] * r_mult
            expected_new *= (new_price / new_sku_base_price) ** price_elasticity
            expected_new *= new_lift
            expected_new *= np.random.lognormal(mean=0, sigma=0.10)
            new_units = sample_sales(expected_new)
        
        # --- Step 4c: Apply cannibalisation (reduce existing SKUs by a share of new_units) ---
        final_sales = raw_sales.copy()
        if new_is_live and new_units > 0:
            for victim_sku, rate in cannibalise_on.items():
                if victim_sku in final_sales:
                    reduced = final_sales[victim_sku] - int(rate * new_units)
                    final_sales[victim_sku] = max(0, reduced)
        
        # --- Step 4d: Write rows for existing SKUs ---
        for sku_name, units in final_sales.items():
            price = sku_prices[sku_name]
            promo_flag = sku_promo_flags[sku_name]
            revenue = round(units * price, 2)
            rows.append({
                "Date": date,
                "Region": region,
                "Category": category_name,
                "SKU": sku_name,
                "Price": price,
                "Promo": promo_flag,
                "Sales_Qty": units,
                "Revenue": revenue,
                "Launch_Week": 0,         # existing SKUs are available from Week 1
                "Is_New": 0
            })
        
        # --- Step 4e: Write row for NEW SKU if live ---
        if new_is_live:
            revenue_new = round(new_units * new_price, 2)
            rows.append({
                "Date": date,
                "Region": region,
                "Category": category_name,
                "SKU": new_sku_name,
                "Price": new_price,
                "Promo": new_promo,
                "Sales_Qty": new_units,
                "Revenue": revenue_new,
                "Launch_Week": new_sku["launch_week"],
                "Is_New": 1
            })

In [ ]:
# ===============================
# STEP 5 — Assemble & Save
# ===============================
df = pd.DataFrame(rows)
df = df.sort_values(["Date", "Region", "SKU"]).reset_index(drop=True)

# Optional: sanity checks
print("Rows:", len(df))
print(df.head(10))

# Save to CSV
out_path = "dairy_milk_cannibalisation_20w.csv"
df.to_csv(out_path, index=False)
print(f"\nSaved: {out_path}")

Rows: 273
        Date    Region       Category                         SKU  Price  \
0 2024-01-07   England  Chocolate Bar       Dairy Milk Almond 45g   1.19   
1 2024-01-07   England  Chocolate Bar      Dairy Milk Caramel 45g   1.07   
2 2024-01-07   England  Chocolate Bar      Dairy Milk Classic 45g   1.09   
3 2024-01-07   England  Chocolate Bar  Dairy Milk Fruit & Nut 45g   1.39   
4 2024-01-07  Scotland  Chocolate Bar       Dairy Milk Almond 45g   1.19   
5 2024-01-07  Scotland  Chocolate Bar      Dairy Milk Caramel 45g   1.29   
6 2024-01-07  Scotland  Chocolate Bar      Dairy Milk Classic 45g   1.09   
7 2024-01-07  Scotland  Chocolate Bar  Dairy Milk Fruit & Nut 45g   1.39   
8 2024-01-07     Wales  Chocolate Bar       Dairy Milk Almond 45g   1.19   
9 2024-01-07     Wales  Chocolate Bar      Dairy Milk Caramel 45g   1.29   

   Promo  Sales_Qty  Revenue  Launch_Week  Is_New  
0      0        112   133.28            0       0  
1      1        218   233.26            0       0

In [ ]:
# ===============================
# STEP 6 — Quick pivots to eyeball cannibalisation
# ===============================
# Pivot total sales by week for Classic vs NEW (all regions combined)
weekly = (
    df.groupby(["Date", "SKU"], as_index=False)["Sales_Qty"]
      .sum()
      .pivot(index="Date", columns="SKU", values="Sales_Qty")
      .fillna(0)
)
print("\nWeekly total units (Classic vs NEW):")
cols_to_show = [c for c in weekly.columns if c in ["Dairy Milk Classic 45g", "NEW Dairy Milk Crunch 45g"]]
print(weekly[cols_to_show].tail(12))


Weekly total units (Classic vs NEW):
SKU         Dairy Milk Classic 45g
Date                              
2024-03-03                   684.0
2024-03-10                   630.0
2024-03-17                   573.0
2024-03-24                   818.0
2024-03-31                   570.0
2024-04-07                   505.0
2024-04-14                   475.0
2024-04-21                   719.0
2024-04-28                   445.0
2024-05-05                   636.0
2024-05-12                   532.0
2024-05-19                   367.0
